### Overfitting 완화 전체 코드 - 2번 페이지

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error

# 1. 데이터 로드
train_df = pd.read_csv("downloads/train.csv")
test_df = pd.read_csv("downloads/test.csv")
submission_df = pd.read_csv("downloads/sample_submission.csv")

# test.csv에서 원본 id 저장
test_ids = test_df["id"].copy()
test_df = test_df.drop(columns=["id"])  # 이후 분석을 위해 삭제

In [ ]:
# 2. 데이터 전처리
# 'Sex' 라벨 인코딩
label_encoder = LabelEncoder()
train_df["Sex"] = label_encoder.fit_transform(train_df["Sex"])
test_df["Sex"] = label_encoder.transform(test_df["Sex"])

# Height가 0인 경우 평균값으로 대체
height_mean = train_df.loc[train_df["Height"] > 0, "Height"].mean()
train_df.loc[train_df["Height"] == 0, "Height"] = height_mean
test_df.loc[test_df["Height"] == 0, "Height"] = height_mean

# 새로운 특성 추가
train_df["Volume"] = train_df["Length"] * train_df["Diameter"] * train_df["Height"]
test_df["Volume"] = test_df["Length"] * test_df["Diameter"] * test_df["Height"]

train_df["Shucked Weight Ratio"] = train_df["Shucked Weight"] / train_df["Weight"]
test_df["Shucked Weight Ratio"] = test_df["Shucked Weight"] / test_df["Weight"]

train_df["Viscera Weight Ratio"] = train_df["Viscera Weight"] / train_df["Weight"]
test_df["Viscera Weight Ratio"] = test_df["Viscera Weight"] / test_df["Weight"]

train_df["Density"] = train_df["Weight"] / train_df["Volume"]
test_df["Density"] = test_df["Weight"] / test_df["Volume"]

train_df["Edible Weight Ratio"] = (train_df["Shucked Weight"] + train_df["Viscera Weight"]) / train_df["Weight"]
test_df["Edible Weight Ratio"] = (test_df["Shucked Weight"] + test_df["Viscera Weight"]) / test_df["Weight"]

# IQR 기반 이상치 제거 함수 정의 및 적용
def remove_outliers_iqr(df, cols, threshold=3.0):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return df[~((df[cols] < lower_bound) | (df[cols] > upper_bound)).any(axis=1)]

num_cols = train_df.select_dtypes(include=["float64"]).columns
train_df = remove_outliers_iqr(train_df, num_cols, threshold=3.0)

# 중복 데이터 제거 및 'id' 컬럼 삭제
train_df = train_df.drop_duplicates().drop(columns=["id"])

# X, y 분리
X = train_df.drop(columns=["Age"])
y = train_df["Age"]

# Train/Validation Split (8:2 비율)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
# 3. 스케일링 (정규화)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [ ]:
# 4. GridSearch를 이용한 최적 하이퍼파라미터 찾기 (교차 검증 cv=5)
param_grid = {
    "n_estimators": [200, 500],
    "max_depth": [5, 10, 15, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [4, 6, 8, 10]
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="neg_mean_absolute_error", n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
print("Best Parameters:", best_params)

In [ ]:
# 5. 최적 하이퍼파라미터로 모델 학습
best_rf = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
best_rf.fit(X_train_scaled, y_train)

# 6. Feature Importance 분석하여 중요도가 0.01 이하인 특성 제거
feature_importances = best_rf.feature_importances_
feature_names = X.columns

# 중요도가 0.01 이하인 특성 찾기
low_importance_features = [feature_names[i] for i in range(len(feature_importances)) if feature_importances[i] < 0.01]
print("Removing Features with Importance < 0.01:", low_importance_features)

# 중요도가 낮은 특성 제거
X_train = X_train.drop(columns=low_importance_features)
X_valid = X_valid.drop(columns=low_importance_features)
test_df = test_df.drop(columns=low_importance_features)

# 다시 스케일링
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
test_scaled = scaler.transform(test_df)

# 7. 다시 모델 학습 및 예측
best_rf.fit(X_train_scaled, y_train)
y_pred = best_rf.predict(X_valid_scaled)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Validation MAE: {mae:.4f}")

In [ ]:
# 8. 테스트 데이터 예측
test_preds = best_rf.predict(test_scaled)

# 9. 제출 파일 생성 (원래 id 유지)
submission = pd.DataFrame({"id": test_ids, "Age": np.round(test_preds, 3)})
submission.to_csv("downloads/sample_submission.csv", index=False)
print("sample_submission.csv 파일 생성 완료!")